In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/playground-series-s3e26/train.csv", index_col='id')
test_df = pd.read_csv("/kaggle/input/playground-series-s3e26/test.csv", index_col='id')

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import log_loss, make_scorer
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler

In [4]:
def days_to_years(df, columns):
    return df[columns].apply(lambda x: x / 365.25)


def make_submission(model, X):
#     test_df[['N_Days', 'Age']] = days_to_years(test_df, ['N_Days', 'Age'])
#     test_num_features = test_df.select_dtypes(np.number)
#     test_cat_features = test_df.select_dtypes('object')

#     encoded_features = pd.concat([test_num_features,
#                                   pd.get_dummies(test_cat_features, dtype='int32')], axis=1)
    y_pred = model.predict_proba(X)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = ['Status_C', 'Status_CL','Status_D']

    submission_df = pd.DataFrame()
    submission_df = y_pred  
    submission_df['id'] = X.index
    submission_df['id'] = submission_df['id'].apply(lambda x : int(x))
    return submission_df
#     submission_df.to_csv('submission.csv', index= False)

In [5]:
train_df[['N_Days', 'Age']] = days_to_years(train_df, ['N_Days', 'Age'])
test_df[['N_Days', 'Age']] = days_to_years(test_df, ['N_Days', 'Age'])

In [6]:
train_data = train_df.copy()
train_labels = train_data['Status'].map({'C':0, 'CL':1, 'D':2})
train_data = train_data.drop('Status', axis=1)

train_num_cols = train_data.select_dtypes(np.number)
train_cat_cols = train_data.select_dtypes('object')

train_data = pd.concat([train_num_cols, pd.get_dummies(train_cat_cols, dtype='int32'), train_labels], axis=1)



test_data = test_df.copy()

test_num_cols = test_data.select_dtypes(np.number)
test_cat_cols = test_data.select_dtypes('object')

test_data = pd.concat([test_num_cols, pd.get_dummies(test_cat_cols, dtype='int32')], axis=1)



In [7]:
# %pip install h2o

In [8]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()
train = h2o.H2OFrame(train_data)
# test = h2o.H2OFrame(test_data)

label ='Status'
columns = train.columns
columns.remove(label)

aml = H2OAutoML(max_models=30, max_runtime_secs=1000)
train[label] = train[label].asfactor()
# aml.train(x=columns, y=label, training_frame=train)

# model = aml.leader
# preds = model.predict(test)
# preds_df = h2o.as_list(preds)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8vic3f5g
  JVM stdout: /tmp/tmp8vic3f5g/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp8vic3f5g/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_unknownUser_j2a3db
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
# test_data = test_df.copy()

# test_num_cols = test_data.select_dtypes(np.number)
# test_cat_cols = test_data.select_dtypes('object')

# test_data = pd.concat([test_num_cols, pd.get_dummies(test_cat_cols, dtype='int32')], axis=1)


# test_data
# model = xgb.XGBClassifier()
# model.fit(train_data, train_labels)
# print(preds.head())
# make_submission(model, test_data)



# preds_df = preds_df.drop('predict', axis=1)
# preds_df['id'] = test_df.index
# preds_df.columns = ['Status_C', 'Status_CL','Status_D', 'id']




In [10]:
# preds_df.to_csv('submission.csv', index= False)

In [11]:
# h2o.explain(model, train)